# * ETL-Actual MKS : Broadband
    (P, G, H, HH, CCAA, CCAATT, Z) level

## Parameter

In [5]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re
import FN_Actual_MKS_Broadband as fn

config = configparser.ConfigParser()
config.read('../../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
next_week_dt = curr_dt + dt.timedelta(days=7)
str_curr_dt = curr_dt.strftime('%Y%m%d')
str_next_week_dt = next_week_dt.strftime('%Y%m%d')
yyyymm_curr = curr_dt.strftime('%Y%m')
yyyymm_next_week = next_week_dt.strftime('%Y%m')

### Lasted Source Summary

In [7]:
# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()


try:
    # Get : Actual Summary from "FCT_BB_SHARE_SUBS_CCAATT"
    src_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(SUBS) TOTAL
            , SUM(CASE WHEN ISP = 'TOL' THEN SUBS END) TOL
            , SUM(CASE WHEN ISP = '3BB' THEN SUBS END) "3BB"
            , SUM(CASE WHEN ISP = 'AIS' THEN SUBS END) AIS
            , SUM(CASE WHEN ISP IN ('CAT', 'TOT') THEN SUBS END) NT
        FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT A
        WHERE TM_KEY_MTH >= 202401
        --WHERE TM_KEY_MTH = (SELECT MAX(TM_KEY_MTH) FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT NOLOCK)
        AND EXISTS (SELECT 1 FROM CDSAPPO.DIM_MOOC_AREA O
			        WHERE O.REMARK <> 'Dummy'
			        AND O.CCAATT = A.CCAATT)
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = src_cur.fetchall()
    print(f'\nCurrent Source Summary...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    tmp_src_df = chk_src_df.copy()
    mod_col_list = tmp_src_df.iloc[:, 1:6].columns.tolist()
    for col in mod_col_list:
        tmp_src_df[col] = tmp_src_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{tmp_src_df}')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Current Source Summary...

   TM_KEY_MTH      TOTAL        TOL        3BB        AIS         NT
0      202401  8,392,163  3,062,361  2,206,741  1,703,305  1,419,756
1      202402  8,399,105  3,065,282  2,197,017  1,716,028  1,420,779
2      202403  8,414,203  3,072,991  2,187,215  1,732,168  1,421,829
3      202404  8,422,602  3,073,168  2,177,393  1,749,183  1,422,858
4      202405  8,439,795  3,082,458  2,167,565  1,765,884  1,423,888
5      202406  8,454,105  3,089,185  2,157,762  1,782,241  1,424,918
6      202407  8,472,239  3,099,541  2,147,926  1,798,817  1,425,955
7      202408  8,483,181  3,103,274  2,138,109  1,814,805  1,426,992
8      202409  8,495,250  3,112,071  2,128,334  1,826,827  1,428,018


### Lasted Fact Summary

In [8]:
# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()


try:
    # Get : Actual Summary from "FCT_BROADBAND_MKS"
    tgt_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(CASE WHEN METRIC_CD IN ('VIN00025', 'VIN00026', 'VIN00027', 'VIN00028') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00025' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00026' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00027' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00028' THEN METRIC_VALUE END) NT
            --, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_MTH >= 202401 --AND TM_KEY_MTH <= 202404
        AND TM_KEY_DAY LIKE '%01'
        AND AREA_TYPE = 'P'
        AND REMARK IS NULL
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = tgt_cur.fetchall()
    print(f'\nCurrent Fact Summary...')
    chk_tgt_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])

    # Display
    tmp_tgt_df = chk_tgt_df.copy()
    mod_col_list = tmp_tgt_df.iloc[:, 1:6].columns.tolist()
    for col in mod_col_list:
        tmp_tgt_df[col] = tmp_tgt_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{tmp_tgt_df}')

    # Get : MAX(TM_KEY_MTH)
    tgt_cur.execute("SELECT MAX(TM_KEY_MTH) FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK")
    max_mth_fct = tgt_cur.fetchone()
    print(f'\nv_max_mth_fct: {max_mth_fct[0]}')
    
    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()


Current Fact Summary...

   TM_KEY_MTH      TOTAL        TOL        3BB        AIS         NT
0      202401  8,392,868  3,062,361  2,206,741  1,703,305  1,420,462
1      202402  8,399,793  3,065,282  2,197,017  1,716,028  1,421,467
2      202403  8,414,874  3,072,991  2,187,215  1,732,168  1,422,500
3      202404  8,423,261  3,073,168  2,177,396  1,749,183  1,423,514
4      202405  8,440,448  3,082,458  2,167,576  1,765,884  1,424,531
5      202406  8,454,751  3,089,185  2,157,779  1,782,241  1,425,547
6      202407  8,472,875  3,099,541  2,147,945  1,798,817  1,426,571
7      202408  8,483,806  3,103,274  2,138,130  1,814,805  1,427,596
8      202409  8,495,860  3,112,071  2,128,357  1,826,827  1,428,606

v_max_mth_fct: 202410


### Check Diff Summary

In [9]:
chk_diff_df = chk_src_df.iloc[-1].compare(chk_tgt_df.iloc[-1])
chk_diff_df['diff'] = chk_diff_df['self'] - chk_diff_df['other']
mod_col_list = chk_diff_df.columns.tolist()
for col in mod_col_list:
    chk_diff_df[col] = chk_diff_df[col].apply(lambda x: format(x, ',.0f'))

chk_diff_df

,self,other,diff
TOTAL,"8,495,250","8,495,860",-610
3BB,"2,128,334","2,128,357",-22
NT,"1,428,018","1,428,606",-588


### Input Parameter

In [10]:
# Manual Config
v_target_schema = 'AUTOKPI'
v_target_table = 'FCT_BROADBAND_MKS'
v_sql_update_fact = 'Update-FCT_BROADBAND_MKS.sql'
v_sql_mockup_fact = 'Mock-FCT_BROADBAND_MKS.sql'
v_sql_initial_fact = 'Initial-FCT_BROADBAND_MKS.sql'


# Auto Config
v_update_flag = 'Y' if chk_diff_df.size > 0 else 'N'
v_mth_end_src = chk_src_df['TM_KEY_MTH'].max().astype(float)
v_mth_end_fct = chk_tgt_df['TM_KEY_MTH'].max().astype(float)
v_max_mth_fct = max_mth_fct[0]
v_mth_end_fct_year = pd.to_datetime(v_mth_end_fct, format='%Y%m').year
v_mth_end_fct_month = pd.to_datetime(v_mth_end_fct, format='%Y%m').month
v_prev_mth_fct = v_mth_end_fct-1 if v_mth_end_fct_month != 1 else float(str(v_mth_end_fct_year-1)+str(12))


# Show Auto Config
print(f'\nyyyymm_next_week: {yyyymm_next_week}')
print(f'\nv_update_flag: {v_update_flag}')
print(f'v_mth_end_src: {v_mth_end_src}')
print(f'v_mth_end_fct: {v_mth_end_fct}')
print(f'v_max_mth_fct: {v_max_mth_fct}')
print(f'v_prev_mth_fct: {v_prev_mth_fct}')


yyyymm_next_week: 202410

v_update_flag: Y
v_mth_end_src: 202409.0
v_mth_end_fct: 202408.0
v_max_mth_fct: 202409
v_prev_mth_fct: 202407.0


## ETL Process...

### DB source to DB Target
    Delete -> Insert

    Source : CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT
             CDSAPPO.DIM_MOOC_AREA
             CDSAPPO.DIM_TIME
    
    Target : AUTOKPI.FCT_BROADBAND_MKS

#### Incremental

In [11]:
''' Auto Process '''

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

# Process flow
if v_update_flag == 'Y':
    if yyyymm_next_week > str(v_mth_end_fct):
        print(f'\n*** Source update & mockup to next month Fact ***')
        fn.src_update_to_fact(v_mth_end_fct, v_target_schema, v_target_table, v_sql_update_fact)
    else:
        print(f'\n*** Source update current month Fact ***')
        fn.src_update_to_fact(v_prev_mth_fct, v_target_schema, v_target_table, v_sql_update_fact)

elif v_update_flag == 'N':
    if yyyymm_next_week > str(v_max_mth_fct):
        print(f'\n*** Last Fact mockup to next month ***')
        fn.mockup_to_fact(v_max_mth_fct, v_target_schema, v_target_table, v_sql_mockup_fact)
    else:
        print(f'\n*** Not update ***')

print(f'\nJob Done !!!')


Job Start... 2024-10-08, 14:41:56

*** Source update & mockup to next month Fact ***

Param input...

   -> mth_end_fct: 202408.0
   -> target_schema: AUTOKPI
   -> target_table: FCT_BROADBAND_MKS
   -> sql_update_fact: Update-FCT_BROADBAND_MKS.sql
   -> v_query_param: {'mth_end_fct': 202408.0}

TDMDBPR : Connected

AKPIPRD : Connected

Processing...

Create Dataframe...

   -> src_df : 5210010 rows, 16 columns

   -> DELETE : "FCT_BROADBAND_MKS" : Done !

   -> INSERT : "FCT_BROADBAND_MKS" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


#### Initial

In [6]:
''' Initial Process '''

# Input Period
# v_initial_mth_start = 202401
# v_initial_mth_end = 202406

# job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# print(f'\nJob Start... {job_start_datetime}')

# print(f'\n*** Re-run initial data to Fact ***')
# fn.src_initial_to_fact(v_initial_mth_start, v_initial_mth_end, v_target_schema, v_target_table, v_sql_initial_fact)

# print(f'\nJob Done !!!')


Job Start... 2024-09-05, 16:06:57

*** Re-run initial data to Fact ***

Param input...

   -> initial_mth_start: 202401
   -> initial_mth_end: 202406
   -> target_schema: AUTOKPI
   -> target_table: FCT_BROADBAND_MKS
   -> sql_initial_fact: Initial-FCT_BROADBAND_MKS.sql
   -> v_query_param: {'initial_mth_start': 202401, 'initial_mth_end': 202406}

TDMDBPR : Connected

AKPIPRD : Connected

Processing...

Create Dataframe...

   -> src_df : 15544620 rows, 16 columns

   -> DELETE : "FCT_BROADBAND_MKS" : Done !

   -> INSERT : "FCT_BROADBAND_MKS" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


## Check Result : "FCT_BROADBAND_MKS"

In [12]:
''' Create Result DataFrame '''

# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
tgt_cur = tgt_conn.cursor()
# print(f'\n{AKPIPRD_db} : Connected')


try:
    # Get : Result Data Summary
    tgt_cur.execute("""
        SELECT TM_KEY_MTH, AREA_NO, AREA_TYPE, COALESCE(REMARK, 'Actual') REMARK
            -->> Subs
            , SUM(CASE WHEN METRIC_CD IN ('VIN00025', 'VIN00026', 'VIN00027', 'VIN00028') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00024' THEN METRIC_VALUE END) "AIS & 3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00025' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00026' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00027' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00028' THEN METRIC_VALUE END) NT
            -->> % MKS
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD IN ('VIN00020', 'VIN00021', 'VIN00022', 'VIN00023') THEN METRIC_VALUE END) "% TOTAL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00019' THEN METRIC_VALUE END) "% AIS & 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00020' THEN METRIC_VALUE END) "% TOL"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00021' THEN METRIC_VALUE END) "% 3BB"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00022' THEN METRIC_VALUE END) "% AIS"
            , SUM(CASE WHEN AREA_TYPE = 'P' AND METRIC_CD = 'VIN00023' THEN METRIC_VALUE END) "% NT"
            -->> Count
            , COUNT(DISTINCT METRIC_CD) CNT_METRIC, COUNT(1) ROW_CNT, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_DAY LIKE '%01'
        GROUP BY TM_KEY_MTH, AREA_NO, AREA_TYPE, REMARK
        ORDER BY 1,2
    """)
    rows = tgt_cur.fetchall()
    print(f'\nGet : Fact Summary...')
    chk_result_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])
    print(f'\n   -> chk_result_df : {chk_result_df.shape[0]} rows, {chk_result_df.shape[1]} columns') 
    
    # Display
    tmp_result_df = chk_result_df.copy()
    # tmp_result_df = tmp_result_df.replace(np.nan, None)
    # tmp_result_df.iloc[:, 4:18] = tmp_result_df.iloc[:, 4:18].fillna(0)
    mod_col_list = tmp_result_df.iloc[:, 4:18].columns.tolist()
    for col in mod_col_list:
        tmp_result_df[col] = tmp_result_df[col].apply(lambda x: format(x, ',.2f') if re.search('%', col) else format(x, ',.0f'))
        
    tgt_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()


Get : Fact Summary...

   -> chk_result_df : 90 rows, 19 columns


In [13]:
''' Group by '''

agg_df = chk_result_df.groupby(['REMARK', 'TM_KEY_MTH', 'CNT_METRIC']).agg({'AREA_TYPE': 'count', 'ROW_CNT': 'sum', 'LOAD_DATE': 'max'}).reset_index()
agg_df

,REMARK,TM_KEY_MTH,CNT_METRIC,AREA_TYPE,ROW_CNT,LOAD_DATE
0,Actual,202401,10,9,85410,2024-09-05 16:41:58.429323
1,Actual,202402,10,9,85410,2024-09-05 16:41:58.429323
2,Actual,202403,10,9,85410,2024-09-05 16:41:58.429323
3,Actual,202404,10,9,85410,2024-09-05 16:41:58.429323
4,Actual,202405,10,9,85410,2024-09-05 16:41:58.429323
5,Actual,202406,10,9,85410,2024-09-05 16:41:58.429323
6,Actual,202407,10,9,85410,2024-09-05 17:06:27.888994
7,Actual,202408,10,9,85410,2024-09-11 15:10:30.149717
8,Actual,202409,10,9,85410,2024-10-08 14:53:53.742582
9,Data as of : 202409,202410,10,9,85410,2024-10-08 14:53:53.742582


In [14]:
''' Reconcile '''

# Filter
max_mth = chk_result_df['TM_KEY_MTH'].max()
# rec_df = tmp_result_df.loc[tmp_result_df['TM_KEY_MTH']==max_mth]
rec_df = tmp_result_df.loc[tmp_result_df['AREA_TYPE']=='P']
rec_df = rec_df.loc[rec_df['TM_KEY_MTH']>=202401]
rec_df = rec_df.reset_index(drop=True)

rec_df#.tail(3)

,TM_KEY_MTH,AREA_NO,AREA_TYPE,REMARK,TOTAL,AIS & 3BB,TOL,3BB,AIS,NT,% TOTAL,% AIS & 3BB,% TOL,% 3BB,% AIS,% NT,CNT_METRIC,ROW_CNT,LOAD_DATE
0,202401,1,P,Actual,"8,392,868","3,910,046","3,062,361","2,206,741","1,703,305","1,420,462",100.00,46.59,36.49,26.29,20.29,16.92,10,10,2024-09-05 16:41:58.429323
1,202402,1,P,Actual,"8,399,793","3,913,045","3,065,282","2,197,017","1,716,028","1,421,467",100.00,46.59,36.49,26.16,20.43,16.92,10,10,2024-09-05 16:41:58.429323
2,202403,1,P,Actual,"8,414,874","3,919,383","3,072,991","2,187,215","1,732,168","1,422,500",100.00,46.58,36.52,25.99,20.58,16.90,10,10,2024-09-05 16:41:58.429323
3,202404,1,P,Actual,"8,423,261","3,926,578","3,073,168","2,177,396","1,749,183","1,423,514",100.00,46.62,36.48,25.85,20.77,16.90,10,10,2024-09-05 16:41:58.429323
4,202405,1,P,Actual,"8,440,448","3,933,460","3,082,458","2,167,576","1,765,884","1,424,531",100.00,46.60,36.52,25.68,20.92,16.88,10,10,2024-09-05 16:41:58.429323
5,202406,1,P,Actual,"8,454,751","3,940,019","3,089,185","2,157,779","1,782,241","1,425,547",100.00,46.60,36.54,25.52,21.08,16.86,10,10,2024-09-05 16:41:58.429323
6,202407,1,P,Actual,"8,472,875","3,946,763","3,099,541","2,147,945","1,798,817","1,426,571",100.00,46.58,36.58,25.35,21.23,16.84,10,10,2024-09-05 17:06:27.888994
7,202408,1,P,Actual,"8,483,806","3,952,936","3,103,274","2,138,130","1,814,805","1,427,596",100.00,46.59,36.58,25.20,21.39,16.83,10,10,2024-09-11 15:10:30.149717
8,202409,1,P,Actual,"8,495,860","3,955,183","3,112,071","2,128,357","1,826,827","1,428,606",100.00,46.55,36.63,25.05,21.50,16.82,10,10,2024-10-08 14:53:53.742582
9,202410,1,P,Data as of : 202409,"8,495,860","3,955,183","3,112,071","2,128,357","1,826,827","1,428,606",100.00,46.55,36.63,25.05,21.50,16.82,10,10,2024-10-08 14:53:53.742582
